## Critical path showcase



This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

The latter method is recommended as the dependencies found with `from simpy` correspond wth the events as scheduled within the simulation, whereas `from recorded activities` assumes dependencies from recorded data (dependencies assumed in case of identical timestamps and simulation objects).

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (third) vessel may move the amount from site 1 to site 2.
* four barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (fifth) vessel moves the entire amount from site 1 to site 2.

Note that the barges have a start event (delay) and velocity based on their number (i.e. barge 0 has zero delay, barge1 100seconds, barge2 200 seconds et cetera).


#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, AlteredStepEnv
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 (nr resources=1) to site 2 (nr resources=4) whereafter another vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration - 200*i
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":i*100,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 2. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [3]:
# Run the simulation.
simulation_data_out = demo_data(2, 100, env=AlteredStepEnv)

In [4]:
# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,9e2e696c-ccf1-471e-8593-b36054f2c37e,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,a71cf5da-e3fc-4d07-acd3-c9f1e614d3bd,False
9,1667f186-14c6-4902-91c6-925aab08d07f,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,aff27bf4-b01a-46e3-a882-ce2435ba11e3,True
15,3b785e5d-7a3c-496e-8eda-c518c03d5311,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,e37e16d5-27eb-4ae4-811a-e31d6da9f1a9,False
18,802ffdb9-a973-4dfe-90ce-cabcdb4cdf5e,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,3f6a73be-0857-42f6-a518-515878cf974d,True
21,5503bae3-ebea-442f-b8bd-620765e5553c,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,99b11d0e-2e00-424a-86c4-61ae7f199d00,True


In [5]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.get_plotly_plot()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [6]:
# Run the simulation.
simulation_data_out = demo_data(2, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,9e4ea05a-d739-4179-81d0-6097ec142c29,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,c7295d57-f58f-4b52-8cea-7e56e52b3ccc,False
9,f92fbe8e-f0ca-4560-b257-e5c3c91a3af0,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,94da86a4-d0d0-4468-ad4c-ddb21cb78cfa,False
15,a1060e09-957e-487b-bf45-ca2fb3e117a7,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,e4b1c805-ffc6-4e3a-a6c1-f7ad049e7ba3,False
18,be0acc5d-b1dd-4ef1-9716-5106f6bb395e,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,2b839ede-a1e2-4128-9bea-213f21a8a8ed,False
21,56c52a36-d442-439a-8645-348fc215a12b,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,1fe50e4b-0050-4627-8f65-4bfa8dd44426,False


In [7]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.get_plotly_plot()

#### 4. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 4 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [8]:
# Run the simulation.
simulation_data_out = demo_data(4, 100, env=AlteredStepEnv)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,56a9a3bd-d190-435a-92e7-8458f752c4d1,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,b450fba2-b567-4acd-a743-c4cc5d9ad877,False
11,c85a44ce-b670-4e6c-8aa4-a2f5ad1173c7,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,33898195-14ed-4d35-8f65-4413a688bae7,False
17,da979001-91ac-4151-8d71-28dca05780ae,basic activity:barge_2,barge_2,1970-01-01 00:03:20,1970-01-01 00:36:40,0 days 00:33:20,ACTIVE,352ce53a-9957-4e46-98cb-d7ad37b51bb1,False
23,72cb0556-ab69-46b3-ba92-638ebcaa5711,basic activity:barge_3,barge_3,1970-01-01 00:05:00,1970-01-01 00:38:20,0 days 00:33:20,ACTIVE,9b5064e4-5d64-4505-a740-a52709030367,True
29,0052d45b-a899-4a25-8ac6-5348f9c38ab2,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,fc187fa8-0f1d-46ff-9b41-dacd426a4a50,False


In [9]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.get_plotly_plot()